# COWER: 2025 Edition: Runs for Fixed-Bottom & Floating Offshore Wind

National Renewable Energy Laboratory\
Daniel Mulas Hernando\
21 November 2025\
WOMBAT version of originally published notebook: 0.13

This notebook serves as a demonstration of the default data fixed-bottom and floating offshore wind data, and
offers a reproducible workflow for the OpEx results for the fixed-bottom and floating offshore wind farms presented in the Cost of Wind Energy Review: 2025 Edition.

As such, these data were produced as a part of the annual Cost of Wind Energy Review (COWER) analysis project.
Similarly, this notebook, and the underlying data will represent the work and updates for the current
year's analysis work.

This notebook allows you to specify the random seeds—and thus control the number of simulations—to run for this particular case.
For each simulation, it calculates and saves the availability, OpEx, and vessel cost breakdowns.
All results are stored in the `library/default/results` folder for easy access and analysis.


In [1]:
from pathlib import Path

import yaml
import numpy as np
import pandas as pd

from wombat.core import Simulation, Metrics
from wombat.core.library import DEFAULT_DATA


N_RUNS = 50
RANDOM_SEEDS = list(range(1, N_RUNS + 1))

In [2]:
def run_windfarm_simulations(technology: str, random_seeds: list):
    """
    Run simulations for a specified wind farm technology ('floating' or 'fixed_bottom')
    and save availability, OpEx, and vessel results for all runs. Saves CSV files in
    library/default/results with appropriate naming.

    Parameters
    ----------
    technology : str
        One of "floating" or "fixed_bottom".
    random_seeds : list[int]
        Random seeds to use to vary Weibull sampled failures.
    """
    # === CONFIGURATION ===
    results_dir = DEFAULT_DATA / "results"
    if not results_dir.is_dir():
        results_dir.mkdir()

    # Select config file based on technology
    if technology.lower() == "floating":
        config_name = "base_osw_floating.yaml"
    elif technology.lower() == "fixed_bottom":
        config_name = "base_osw_fixed.yaml"
    else:
        raise ValueError("Invalid technology. Must be 'floating' or 'fixed_bottom'.")

    # Initialize storage lists
    availability_records = []
    opex_records = []
    vessel_records = []
    repair_time_records = []

    # === RUN SIMULATIONS SEQUENTIALLY ===
    N = len(random_seeds)
    for i, seed in enumerate(random_seeds, start=1):
        print(
            f"🚀 Running simulation {i}/N ({technology}) with random seed {seed}",
            end="\r",
        )

        # Run simulation
        sim = Simulation(DEFAULT_DATA, config_name, random_seed=seed)
        sim.run(create_metrics=True, save_metrics_inputs=True)

        # Load metrics
        fpath = sim.env.metrics_input_fname.parent
        fname = sim.env.metrics_input_fname.name
        metrics = Metrics.from_simulation_outputs(fpath, fname)

        # === 1. Availability Results ===
        time_avail = metrics.time_based_availability(frequency="project", by="windfarm")
        prod_avail = metrics.production_based_availability(
            frequency="project", by="windfarm"
        )
        time_value = time_avail.iloc[0, 0]
        prod_value = prod_avail.iloc[0, 0]
        availability_records.append(
            {
                "run": i,
                "random_seed": seed,
                "time_based_availability": time_value,
                "production_based_availability": prod_value,
            }
        )

        # === 2. OpEx Results ===
        opex_df = metrics.opex(frequency="annual", by_category=True).reset_index()
        opex_df.insert(0, "random_seed", seed)
        opex_df.insert(0, "run", i)
        opex_records.append(opex_df)

        # === 3. Vessel Costs ===
        vessel_df = metrics.equipment_costs(
            frequency="annual", by_equipment=True
        ).reset_index()
        vessel_df.insert(0, "random_seed", seed)
        vessel_df.insert(0, "run", i)
        vessel_records.append(vessel_df)

        # === 4. Repair Time at Port ===

        # Build full path to config file
        config_path = DEFAULT_DATA / "project" / "config" / config_name

        # Load YAML
        with open(config_path, "r") as f:
            config_data = yaml.safe_load(f)

        # Extract port name
        port_name = config_data.get("port", None)

        if port_name is None:
            raise KeyError(
                f"'port' key not found in {config_path}, can not calculate time at port"
            )

        port_name = port_name.replace(".yaml", "")

        events_df = sim.env.load_events_log_dataframe()
        events_df["duration"] = pd.to_numeric(events_df["duration"], errors="coerce")
        df_port = events_df[events_df["agent"] == port_name]
        total_hours = df_port["duration"].sum()
        simulation_years = sim.env.end_year - sim.env.start_year + 1
        avg_hours_per_year = total_hours / simulation_years
        avg_days_per_year = avg_hours_per_year / 24
        avg_months_per_year = avg_hours_per_year / (24 * 30.4375)

        repair_time_records.append(
            {
                "run": i,
                "random_seed": seed,
                "avg_repair_time_months": avg_months_per_year,
                "avg_repair_time_days": avg_days_per_year,
            }
        )

        # Cleanup logs for this simulation
        sim.env.cleanup_log_files()

    # === COMBINE AND SAVE RESULTS ===
    df_availability = pd.DataFrame(availability_records)
    df_opex = pd.concat(opex_records, ignore_index=True)
    df_vessels = pd.concat(vessel_records, ignore_index=True)
    df_repair_time = pd.DataFrame(repair_time_records)

    df_availability.to_csv(
        results_dir / f"COWER-2025-{technology}_all_availability_results.csv",
        index=False,
    )
    df_opex.to_csv(
        results_dir / f"COWER-2025-{technology}_all_opex_results.csv", index=False
    )
    df_vessels.to_csv(
        results_dir / f"COWER-2025-{technology}_all_vessel_results.csv", index=False
    )
    df_repair_time.to_csv(
        results_dir / f"COWER-2025-{technology}_repair_time_at_port_results.csv",
        index=False,
    )

    print(f"✅ All {technology} simulations complete. Results saved to {results_dir}")

In [3]:
# Run fixed-bottom simulations
run_windfarm_simulations("fixed_bottom", random_seeds=RANDOM_SEEDS)

# Run floating simulations
run_windfarm_simulations("floating", random_seeds=RANDOM_SEEDS)

✅ All fixed_bottom simulations complete. Results saved to C:\WOMBAT_COWER_2025\WOMBAT\library\default\results
✅ All floating simulations complete. Results saved to C:\WOMBAT_COWER_2025\WOMBAT\library\default\results


## Summarize Results from Multiple Simulations in One Table

In [3]:
def summarize_simulation(
    library_path=DEFAULT_DATA / "results",
    project_capacity_mw=600,
):
    """
    Compute overall average and standard deviation results per technology (fixed_bottom, floating),
    averaging over all years and all simulation runs, returning a formatted DataFrame
    with monetary values in $/kW-yr, availability in %, and a Units column.

    Parameters
    ----------
    library_path : str or Path
        Path to the folder containing the CSV results.
    project_capacity_mw : float
        Project capacity in MW to normalize costs to $/kW. Default is 600 MW.

    Returns
    -------
    df_summary : pd.DataFrame
        Formatted, transposed DataFrame with categories as rows and columns as technologies,
        including Units column, values rounded to 1 decimal, 0 replaced with NaN, NaN displayed as "-".
        Vessel types are shown as indented subcategories of Equipment Cost.
        Columns for mean and standard deviation per technology are included.
    """
    summary_dict = {}
    capacity_kw = project_capacity_mw * 1_000  # convert MW to kW

    # Step 1: Identify all vessel columns across both technologies
    vessel_cols_all = set()
    for tech in ["fixed_bottom", "floating"]:
        df_vessels = pd.read_csv(
            Path(library_path) / f"COWER-2025-{tech}_all_vessel_results.csv"
        )
        vessel_cols = [
            c for c in df_vessels.columns if c not in ["run", "random_seed", "year"]
        ]
        vessel_cols_all.update(vessel_cols)
    vessel_cols_all = sorted(vessel_cols_all)

    for tech in ["fixed_bottom", "floating"]:
        # Load CSVs
        df_avail = pd.read_csv(
            Path(library_path) / f"COWER-2025-{tech}_all_availability_results.csv"
        )
        df_opex = pd.read_csv(
            Path(library_path) / f"COWER-2025-{tech}_all_opex_results.csv"
        )
        df_vessels = pd.read_csv(
            Path(library_path) / f"COWER-2025-{tech}_all_vessel_results.csv"
        )

        # --- Average and std availability over all runs and years ---
        summary_dict.setdefault("avg_time_based_availability", {})[f"{tech} Mean"] = (
            df_avail["time_based_availability"].mean() * 100
        )
        summary_dict.setdefault("avg_time_based_availability", {})[f"{tech} Std"] = (
            df_avail["time_based_availability"].std() * 100
        )

        summary_dict.setdefault("avg_production_based_availability", {})[
            f"{tech} Mean"
        ] = df_avail["production_based_availability"].mean() * 100
        summary_dict.setdefault("avg_production_based_availability", {})[
            f"{tech} Std"
        ] = df_avail["production_based_availability"].std() * 100

        # --- Average and std OpEx over all runs and years ($/kW-yr) ---
        opex_cols = ["operations", "port_fees", "total_labor_cost", "materials_cost"]
        for col in opex_cols:
            summary_dict.setdefault(col, {})[f"{tech} Mean"] = (
                df_opex[col].mean() / capacity_kw
            )
            summary_dict.setdefault(col, {})[f"{tech} Std"] = (
                df_opex[col].std() / capacity_kw
            )

        # --- Equipment cost as sum of vessels ($/kW-yr) ---
        vessel_total = df_vessels[
            [c for c in vessel_cols_all if c in df_vessels.columns]
        ].sum(axis=1)
        summary_dict.setdefault("equipment_cost", {})[f"{tech} Mean"] = (
            vessel_total.mean() / capacity_kw
        )
        summary_dict.setdefault("equipment_cost", {})[f"{tech} Std"] = (
            vessel_total.std() / capacity_kw
        )

        # --- Individual vessel costs ($/kW-yr) ---
        for col in vessel_cols_all:
            if col in df_vessels.columns:
                summary_dict.setdefault(f"  - {col}", {})[f"{tech} Mean"] = (
                    df_vessels[col].mean() / capacity_kw
                )
                summary_dict.setdefault(f"  - {col}", {})[f"{tech} Std"] = (
                    df_vessels[col].std() / capacity_kw
                )
            else:
                summary_dict.setdefault(f"  - {col}", {})[f"{tech} Mean"] = np.nan
                summary_dict.setdefault(f"  - {col}", {})[f"{tech} Std"] = np.nan

        # --- OpEx total ($/kW-yr) ---
        op_ex_total = (
            df_opex[
                ["operations", "port_fees", "total_labor_cost", "materials_cost"]
            ].sum(axis=1)
            + vessel_total
        )
        summary_dict.setdefault("OpEx_total", {})[f"{tech} Mean"] = (
            op_ex_total.mean() / capacity_kw
        )
        summary_dict.setdefault("OpEx_total", {})[f"{tech} Std"] = (
            op_ex_total.std() / capacity_kw
        )

    # Convert dict to DataFrame
    df_summary = pd.DataFrame(summary_dict).T

    # Reorder rows (🟩 inserted repair time row after availability)
    avail_rows = [
        "avg_time_based_availability",
        "avg_production_based_availability",
    ]
    opex_rows = ["operations", "port_fees", "total_labor_cost", "materials_cost"]
    vessel_rows = ["equipment_cost"] + [f"  - {v}" for v in vessel_cols_all]
    ordered_rows = avail_rows + opex_rows + vessel_rows + ["OpEx_total"]
    df_summary = df_summary.loc[[r for r in ordered_rows if r in df_summary.index]]

    # Capitalize metrics and analysis names, and replace underscores
    df_summary.columns = [col.replace("_", " ").title() for col in df_summary.columns]
    df_summary.index = df_summary.index.str.replace("_", " ").str.title()

    # Add Units column
    units = []
    for idx in df_summary.index:
        if "availability" in idx:
            units.append("%")
        elif "repair" in idx:
            units.append("months / yr")
        else:
            units.append("$ / kW-yr")
    df_summary.insert(0, "Units", units)

    # Replace 0 with NaN and drop empty rows
    df_summary.replace(0, np.nan, inplace=True)
    df_summary.dropna(how="all", inplace=True)

    df_summary.rename(index={"Opex Total": "Total OpEx"}, inplace=True)

    return df_summary

In [4]:
df_summary = summarize_simulation()
df_summary.style.format(na_rep="-", precision=1)

,Units,Fixed Bottom Mean,Fixed Bottom Std,Floating Mean,Floating Std
Avg Time Based Availability,$ / kW-yr,94.3,0.1,92.0,0.3
Avg Production Based Availability,$ / kW-yr,93.9,0.2,92.0,0.4
Operations,$ / kW-yr,47.4,0.1,50.9,0.1
Port Fees,$ / kW-yr,0.5,-,17.4,4.4
Total Labor Cost,$ / kW-yr,-,-,-,-
Materials Cost,$ / kW-yr,2.1,0.6,3.2,1.0
Equipment Cost,$ / kW-yr,73.0,24.9,73.0,17.8
- Anchor Handling Tug,$ / kW-yr,-,-,4.3,2.4
- Cable Laying Vessel,$ / kW-yr,13.4,9.9,20.0,15.2
- Crew Transfer Vessel 1,$ / kW-yr,1.9,0.0,1.9,0.0
